In [ ]:
import numpy as np
import math
import scipy.linalg as la
import matplotlib.pyplot as plt

## Task 1.1

In [ ]:
def RK4step(lamd,told, uold, h): # Denna klassen skrivs just nu där f = skalär*y. 
    Y1 = lamd*uold    #Y1 definieras som f(uold,told).
    Y2 = lamd*(uold + h*Y1/2)
    Y3 = lamd*(uold + h*Y2/2)
    Y4 = lamd*(uold + h*Y3)
    unew = uold + (h/6)*(Y1 + 2*Y2 + 2*Y3 + Y4)

    return unew

def RK4Int(lamd, y0,t0,tf,N):
    h = (tf-t0)/N

    Y = np.zeros((N, 2)) #En matris med vilket steg vi är på samt vilken tid som det motsvarar. 
    Y[0,0] = y0
    Y[0,1] = t0

    for i in range(N-1):
         x = RK4step(lamd, Y[i,1], Y[i,0], h)
         Y[i+1, 0] = x
         Y[i+1, 1] = Y[i,1] + h

    correct = np.zeros_like(Y)
    error = np.zeros_like(Y)
    for i in range(N):
        correct[i,0] = np.exp(lamd*Y[i,1])*y0

    error[:,0] = abs(Y[:,0]-correct[:,0])
    error[:,1] = Y[:,1]

    return Y, error, correct

In [ ]:
def errVSh(A, y0, t0, tf):
    Nvec = [int(np.power(2,i)) for i in range(3, 10)]
    err = np.zeros(len(Nvec))
    for i in range(len(Nvec)):
        Y,error, correct = RK4Int(A, y0, t0, tf, Nvec[i])
        err[i] = error[-1, 0]

    h=np.zeros(len(Nvec))
    for i in range(len(Nvec)):
        h[i] = (tf - t0) /Nvec[i]
    plt.grid()
    plt.xlabel("Stepsize h")
    plt.ylabel("The global error")
    plt.title("The global error as a function of stepsize")
    plt.loglog(h, err, 'r')
    return np.log(err), np.log(h)

In [ ]:
lamd = 0.5 
y0 = 1
t0 = 0 
tf = 1
logerr, logh = errVSh(lamd,y0,t0,tf) #Väldigt linjärt och fint för denna tidsskalan. 
plt.show()
plt.title("Is it O(h^4)?")
plt.xlabel("Log of stepsize")
plt.ylabel("Log of the global error")
plt.grid()
plt.plot(logh,logerr) #Här plottar vi det logarimerade steglängden mot det logaritmerade felet

## Task 1.2-1.4

In [ ]:
def RK34step(f, told, Yold, h):
    Y1 = f(told, Yold)
    Y2 = f(told + h/2, Yold + h/2*Y1)
    Y3 = f(told + h/2, Yold + h/2*Y2)
    Z3 = f(told + h, Yold - h*Y1 + 2*h*Y2)
    Y4 = f(told + h, Yold + h*Y3)
    Ynew = Yold + h/6*(Y1 + 2*Y2 + 2*Y3 + Y4)

    err = np.abs(h/6*(2*Y2+Z3-2*Y3-Y4))

    return Ynew, err #Ser bra ut 

def newstep (tol, err, errold, hold, k):
    hnew = hold * np.power(tol/err, 2/(3*k))*np.power(tol/errold, -1/(3*k))  #Ser bra ut 
    return hnew
    
def adaptiveRK34(f, t0, tf, y0, tol):
    k = 4
    h = (tf-t0)*np.power(tol,1/4)/(100*(1+la.norm(f(t0,y0)))) # Ser bra ut.
    t = [t0]
    Y = [y0]
    ERR = [tol]
    y, err = RK34step(f, t[-1], y0, h)
    Y.append(y) #Begynnelseplatsen (efter y0)
    ERR.append(err) #Begynnelsefelet
    t.append(t[-1] + h)
    h = newstep(tol, ERR[-1], ERR[-2], h, k)  #Ändrade till err[-2] istället för tol. 


    while t[-1] + h < tf:
        y, err = RK34step(f, t[-1], Y[-1], h)
        Y.append(y)
        ERR.append(err)
        t.append(t[-1] + h)
        h = newstep(tol, ERR[-1], ERR[-2], h, k) #Bytte plats på h och t append h. 
       

    y, err = RK34step(f, t[-1], Y[-1], tf-t[-1]) #Ser bra ut 
    t.append(tf)
    Y.append(y)
    ERR.append(err)
    return t, Y, ERR


In [ ]:
y0 = 1
f = lambda t, y: -1.05*y
t, Y, ERR = adaptiveRK34(f, 0, 10, y0, 1e-6)
plt.xlabel("The time")
plt.ylabel("Functionvalue")
plt.title("Our solution to the ODE")
plt.grid()
plt.plot(t, Y, 'b')
print("Amount of steps: ", len(t))


In [ ]:
y0 = 1
f = lambda t, y: -1.05*y
t, Y, ERR = adaptiveRK34(f, 0, 10, y0, 1e-12) # Fett många steg när vi sänker toleransen.
plt.xlabel("The time")
plt.ylabel("Functionvalue")
plt.title("Our solution to the ODE")
plt.grid()
plt.plot(t, Y, 'b')
print("Amount of steps: ", len(t))